In [1]:
import sys
sys.path.append('/app/')  # noqa
sys.path.append('/app/loopa')  # noqa

from jina import Document, DocumentArray, Flow, Executor, requests
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import argparse
from loopa.executors.cache import SBERTEncoderCache
from loopa.executors.index import FaissIndexer, IndexMerger
from loopa.config import CUDA_IF_AVAILABLE
import torch
import hashlib

def my_hash(s): 
    return str(int(hashlib.md5(str(s).encode('utf-8')).hexdigest(), 16))

In [2]:
torch.multiprocessing.set_start_method('spawn', force=True) 

In [3]:
OUT_DIR = Path("/data/imgflip/scrap_language_image_pairs_20220209/")
OUT_TEMPLATE_TEXTS_CSV = OUT_DIR/"template_texts.csv"

In [47]:
JINA_SBERT_EMBEDDING_TEMPLATE_TEXT_COLLECTION = "01_sbert_all_mpnet_base_v2_imgflip_template_100k_embeddings"
MONGO_EMBEDDING_DB_NAME = 'memlish_db'

In [46]:
def text_input_docs(df):    
    for idx, row in df.iterrows():
        template_name = row["template_img_name"]
        try:
            template_text = row["text"]
            document_id = f"{template_name}_{my_hash(template_text)}"
            doc = Document(id=document_id, text=template_text)# tags={'filename': str(template_img_path)})
        except Exception as e:
            print(document_id)
            print(template_text)
            print(str(e))
        
        yield doc 

In [6]:
templates_df = pd.read_csv(OUT_TEMPLATE_TEXTS_CSV)

In [7]:
templates_df = templates_df[~templates_df['text'].isna()]

In [8]:
templates_df = templates_df[~templates_df.duplicated(["text", "template_img_name"])]

In [37]:
templates_df.head(2)

,url,text,alt,views,upvotes,comments,meme_home_page,template_name,template_img_name
0,i.imgflip.com/61ov27.jpg,accepting that ur child made a good point; MOM...,Moms be like | accepting that ur child made a...,7208,106,7,/i/61ov27,Drake-Hotline-Bling,Drake-Hotline-Bling.jpg
1,i.imgflip.com/61soau.jpg,Ground children by sending them to bed early a...,Mexican meme | Ground children by sending the...,5271,65,5,/i/61soau,Drake-Hotline-Bling,Drake-Hotline-Bling.jpg


In [48]:
k_to_take = 100
groups = []

for _, group in templates_df.groupby(['template_img_name']):
    selected_sorted = group.sort_values(by=['views'], ascending=False)[:k_to_take]
    groups.append(selected_sorted)

In [49]:
df = pd.concat(groups).reset_index(drop=True)
df.head(2)

,url,text,alt,views,upvotes,comments,meme_home_page,template_name,template_img_name
0,i.imgflip.com/25p6ij.jpg,"I WON $10,000,000 IN THE LOTTERY AND DECIDED T...","10 Guy | I WON $10,000,000 IN THE LOTTERY AND...",733523,305,107,/i/25p6ij,10-Guy,10-Guy.jpg
1,i.imgflip.com/kbovc.jpg,I NEED TO REFILL MY CHILD REPELLENT,10 Guy | I NEED TO REFILL MY CHILD REPELLENT |...,684545,12,0,/i/kbovc,10-Guy,10-Guy.jpg


In [50]:
embedder_params = {
        "device":CUDA_IF_AVAILABLE
}

cache_params = {
    "embedder_params": embedder_params,
    "db_name": MONGO_EMBEDDING_DB_NAME, 
    "collection_name": JINA_SBERT_EMBEDDING_TEMPLATE_TEXT_COLLECTION,
    "embedding_field_name":'emb',
    "megabatch_size":8192
}

In [51]:
flow_text_encode = Flow().add(uses=SBERTEncoderCache, name="SBERT_encoder", uses_with=cache_params)
flow_text_encode

In [52]:
with flow_text_encode:
    flow_text_encode.post(on='', inputs=text_input_docs(df), request_size=10_000, show_progress=True)

⠏ 1/2 waiting SBERT_encoder to be ready...                                        

Downloading: 100%|██████████| 1.18k/1.18k [00:00<00:00, 1.48MB/s]


⠸ 1/2 waiting SBERT_encoder to be ready...                                        

Downloading: 100%|██████████| 10.1k/10.1k [00:00<00:00, 9.30MB/s]


⠦ 1/2 waiting SBERT_encoder to be ready...                                        

...

⠹ Working... ━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:07:02  0.0 step/s 

  0%|          | 0/1 [00:00<?, ?it/s]

⠦ Working... ━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:07:21  0.0 step/s 

100%|██████████| 1/1 [00:19<00:00, 19.61s/it]


⠧       DONE ━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0:07:21  0.0 step/s 13 steps done in 7 minutes and 21 seconds
